# LangChain Explore

In [27]:
import os
import pandas as pd
from fredapi import Fred # For FRED API
from dotenv import load_dotenv # For loading .env file (optional, for local dev)

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.tools import tool # Updated way to define tools
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
# .env read
load_dotenv()

fred_api_key = os.getenv("FRED_API_KEY")
if not fred_api_key:
    raise ValueError("FRED_API_KEY not found in environment variables. Please set it.")

# Initialize FRED client
fred = Fred(api_key=fred_api_key)

In [ ]:
# fred.stlouisfed.org, data.nasdaq.com/data/FRED-federal-reserve-economic-data

FRED_SERIES_IDS = {
    "Real GDP": "GDPC1",  # Quarterly, Billions of Chained 2017 Dollars
    "Nominal GDP": "GDP", # Quarterly, Billions of Dollars
    "CPI": "CPIAUCSL",  # Monthly, Consumer Price Index for All Urban Consumers: All Items
    "Core CPI": "CPILFESL", # Monthly, Consumer Price Index for All Urban Consumers: Less Food & Energy
    "Unemployment Rate": "UNRATE",  # Monthly, Percent
    "Federal Funds Rate": "DFF",  # Daily (we'll take the latest) -> often reported as FEDFUNDS (monthly avg) for summaries
    "10-Year Treasury": "DGS10", # Daily, 10-Year Treasury Constant Maturity Rate
    "Industrial Production": "INDPRO", # Monthly, Index
    "Personal Saving Rate": "PSAVERT", # Monthly, Percent
    "Retail Sales": "RSAFS", # Monthly, Millions of Dollars
}


@tool
def get_recent_macroeconomic_data(series_names: list[str] = None) -> str:
    """
    Fetches the most recent data for specified U.S. macroeconomic indicators from FRED.
    Valid series_names are: "Real GDP", "Nominal GDP", "CPI", "Core CPI", "Unemployment Rate",
    "Federal Funds Rate", "10-Year Treasury", "Industrial Production", "Personal Saving Rate", "Retail Sales".
    If no series_names are provided, it fetches a default set.
    """
    print(f"--- Tool: get_recent_macroeconomic_data called with series_names: {series_names} ---")

    if not series_names: # Default set if none specified
        series_names = ["Real GDP", "CPI", "Unemployment Rate", "Federal Funds Rate"]

    data_summary = []
    for name in series_names:
        series_id = FRED_SERIES_IDS.get(name)
        if not series_id:
            data_summary.append(f"Warning: Macroeconomic series '{name}' not recognized.")
            continue
        try:
            series_data = fred.get_series(series_id)
            if series_data.empty:
                data_summary.append(f"{name} ({series_id}): No data found.")
                continue

            latest_value = series_data.iloc[-1]
            latest_date = series_data.index[-1].strftime('%Y-%m-%d')

            info = fred.get_series_info(series_id)
            units = info.get('units_short', '')

            data_summary.append(f"{name} ({series_id}) as of {latest_date}: {latest_value:.2f} {units}")

        except Exception as e:
            data_summary.append(f"Error fetching {name} ({series_id}): {str(e)}")

    if not data_summary:
        return "Could not fetch any macroeconomic data."

    return "\n".join(data_summary)

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

tools = [get_recent_macroeconomic_data]

MEMORY_KEY = "chat_history"

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful financial assistant. You have access to a specialized tool that can provide recent U.S. macroeconomic data from FRED."
            "When asked about why a stock or the broader market (like an ETF such as VOO) has moved, or for general economic conditions, "
            "you should consider using the 'get_recent_macroeconomic_data' tool to gather context. "
            "Think about which specific macroeconomic series would be most relevant to the user's question if they don't specify. "
            "Explain your reasoning clearly, combining information from the tool with your general knowledge. "
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY, optional=True),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"), # {agent_scratchpad} is where the agent's intermediate thoughts and tool calls/responses are injected
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True, # to see agent's thought process
    handle_parsing_errors=True # debugging
)

chat_history = []

def ask_agent(user_query):
    global chat_history
    current_chat_history = [msg for msg in chat_history if isinstance(msg, (HumanMessage, AIMessage))]

    response = agent_executor.invoke({
        "input": user_query,
        "chat_history": current_chat_history
    })
    chat_history.append(HumanMessage(content=user_query))
    chat_history.append(AIMessage(content=response["output"]))

    return response["output"]


if __name__ == "__main__":
    print("Financial Agent Initialized. Ask a question (e.g., 'Why might the stock market be down recently?' or 'What's the current unemployment rate?'). \n Type 'exit' to quit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        agent_response = ask_agent(user_input)
        print(f"Agent: {agent_response}\n")

Financial Agent Initialized. Ask a question (e.g., 'Why might the stock market be down recently?' or 'What's the current unemployment rate?'). Type 'exit' to quit.


> Entering new AgentExecutor chain...

Invoking: `get_recent_macroeconomic_data` with `{'series_names': ['CPI', 'Unemployment Rate', 'Federal Funds Rate', 'Retail Sales']}`


--- Tool: get_recent_macroeconomic_data called with series_names: ['CPI', 'Unemployment Rate', 'Federal Funds Rate', 'Retail Sales'] ---
CPI (CPIAUCSL) as of 2025-04-01: 320.32 Index 1982-1984=100
Unemployment Rate (UNRATE) as of 2025-04-01: 4.20 %
Federal Funds Rate (DFF) as of 2025-05-13: 4.33 %
Retail Sales (RSAFS) as of 2025-03-01: 722025.00 Mil. of $
Invoking: `get_recent_macroeconomic_data` with `{'series_names': ['Nominal GDP', 'Core CPI', '10-Year Treasury']}`


--- Tool: get_recent_macroeconomic_data called with series_names: ['Nominal GDP', 'Core CPI', '10-Year Treasury'] ---
Nominal GDP (GDP) as of 2025-01-01: 29977.63 Bil. of $
Core CPI (C

In [ ]:
    # Example questions:
    # response1 = ask_agent("Why might VOO have gone down recently?")
    # print(f"\nAgent Response 1: {response1}")

    # response2 = ask_agent("What's the latest Real GDP and CPI data?")
    # print(f"\nAgent Response 2: {response2}")

    # response3 = ask_agent("How is the job market looking based on recent data?")
    # print(f"\nAgent Response 3: {response3}")

In [44]:
MEMORY_KEY

'chat_history'

In [45]:
chat_history

[HumanMessage(content='Why might VOO have gone down in March?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="In March 2025, several macroeconomic factors could have contributed to a decline in the Vanguard S&P 500 ETF (VOO). Here are some key indicators from that period:\n\n1. **Consumer Price Index (CPI)**: The CPI was reported at 320.32 as of April 1, 2025. This indicates inflationary pressures, which can lead to concerns about rising costs and reduced consumer spending.\n\n2. **Unemployment Rate**: The unemployment rate was at 4.20% as of April 1, 2025. While this is relatively low, any signs of rising unemployment can create uncertainty in the market, affecting investor sentiment.\n\n3. **Federal Funds Rate**: The Federal Funds Rate was at 4.33% as of May 13, 2025. An increase in interest rates can lead to higher borrowing costs, which may slow down economic growth and negatively impact stock prices.\n\n4. **Retail Sales**: Retail sales were reported at $722,025